#BDDA End Term Project
<BR> Answer 2
<BR> By:Aman Manocha
<BR> Roll No: 015001

In [ ]:
# Create a Spark session
from pyspark.sql import SparkSession

from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import stddev
from pyspark.sql.functions import format_number
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression


In [ ]:
spark=SparkSession.builder.appName('Problem 2').getOrCreate()

In [ ]:
# Load dataset
df=spark.read.csv('insurance.csv', header=True, inferSchema=True)

In [ ]:
df.show()

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.13692|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251|
| 23|  male|  34.4|       0|    no|southwest|   1826.843|
| 56|female| 39.82|       0|    no|southeast| 11090.7178|
| 27|  male| 4

In [ ]:
# Columns List
df.columns

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']

In [ ]:
# Columns info
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [ ]:
# Handling Categorical features
indexer = StringIndexer(inputCols=["sex","smoker","region"],outputCols=["sex_indexed","smoker_indexed","region_indexed"])
df1=indexer.fit(df).transform(df)
df1.show()

+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_indexed|smoker_indexed|region_indexed|
+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|        1.0|           1.0|           2.0|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|        0.0|           0.0|           0.0|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|        0.0|           0.0|           0.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|        0.0|           0.0|           1.0|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|        0.0|           0.0|           1.0|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|        1.0|           0.0|           0.0|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|        1.0|           0.0|           0.0|


In [ ]:
df1.columns

['age',
 'sex',
 'bmi',
 'children',
 'smoker',
 'region',
 'charges',
 'sex_indexed',
 'smoker_indexed',
 'region_indexed']

In [ ]:
# Rows having charges above 50000
df1.filter("charges>=50000").show()

+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_indexed|smoker_indexed|region_indexed|
+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+
| 28|  male|  36.4|       1|   yes|southwest|51194.55914|        0.0|           1.0|           2.0|
| 54|female| 47.41|       0|   yes|southeast|63770.42801|        1.0|           1.0|           0.0|
| 31|female|38.095|       1|   yes|northeast|58571.07448|        1.0|           1.0|           3.0|
| 33|female| 35.53|       0|   yes|northwest|55135.40209|        1.0|           1.0|           1.0|
| 60|  male|  32.8|       0|   yes|southwest|52590.82939|        0.0|           1.0|           2.0|
| 52|  male|34.485|       3|   yes|northwest|60021.39897|        0.0|           1.0|           1.0|
| 45|  male| 30.36|       0|   yes|southeast|62592.87309|        0.0|           1.0|           0.0|


In [ ]:
# Count of categories of sex column
df1.groupBy("sex").count().show()

+------+-----+
|   sex|count|
+------+-----+
|female|  662|
|  male|  676|
+------+-----+



In [ ]:
# Grouping region, smoker and sex columns
df1.groupBy("region","smoker","sex").count().show()

+---------+------+------+-----+
|   region|smoker|   sex|count|
+---------+------+------+-----+
|northeast|    no|  male|  125|
|northwest|   yes|female|   29|
|southwest|   yes|female|   21|
|southeast|    no|female|  139|
|southeast|   yes|  male|   55|
|northwest|   yes|  male|   29|
|southwest|    no|  male|  126|
|southeast|   yes|female|   36|
|northwest|    no|  male|  132|
|northwest|    no|female|  135|
|southwest|   yes|  male|   37|
|northeast|   yes|female|   29|
|southwest|    no|female|  141|
|northeast|   yes|  male|   38|
|northeast|    no|female|  132|
|southeast|    no|  male|  134|
+---------+------+------+-----+



In [ ]:
# Maximum charges
df1.agg({"charges":"max"}).show()

+------------+
|max(charges)|
+------------+
| 63770.42801|
+------------+



In [ ]:
#Grouping region
grouped = df1.groupBy("region")

In [ ]:
#Region wise maximum charges
grouped.agg({"charges":"max"}).show()

+---------+------------+
|   region|max(charges)|
+---------+------------+
|northwest| 60021.39897|
|southeast| 63770.42801|
|northeast| 58571.07448|
|southwest| 52590.82939|
+---------+------------+



In [ ]:
# Standard deviation of charges column
charges_std = df1.select(stddev("charges").alias("std"))
charges_std.show()

+------------------+
|               std|
+------------------+
|12110.011236693992|
+------------------+



In [ ]:
# Formatting of number changed
charges_std.select(format_number('std',2)).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|            12,110.01|
+---------------------+



In [ ]:
# Charges column in descending order
df1.orderBy(df["charges"].desc()).show()

+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_indexed|smoker_indexed|region_indexed|
+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+
| 54|female| 47.41|       0|   yes|southeast|63770.42801|        1.0|           1.0|           0.0|
| 45|  male| 30.36|       0|   yes|southeast|62592.87309|        0.0|           1.0|           0.0|
| 52|  male|34.485|       3|   yes|northwest|60021.39897|        0.0|           1.0|           1.0|
| 31|female|38.095|       1|   yes|northeast|58571.07448|        1.0|           1.0|           3.0|
| 33|female| 35.53|       0|   yes|northwest|55135.40209|        1.0|           1.0|           1.0|
| 60|  male|  32.8|       0|   yes|southwest|52590.82939|        0.0|           1.0|           2.0|
| 28|  male|  36.4|       1|   yes|southwest|51194.55914|        0.0|           1.0|           2.0|


In [ ]:
# Missing values
from pyspark.sql.functions import isnan, when, count, col
def get_null_value_count(data):
    data.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in data.columns]).show()

In [ ]:
get_null_value_count(df1)

+---+---+---+--------+------+------+-------+-----------+--------------+--------------+
|age|sex|bmi|children|smoker|region|charges|sex_indexed|smoker_indexed|region_indexed|
+---+---+---+--------+------+------+-------+-----------+--------------+--------------+
|  0|  0|  0|       0|     0|     0|      0|          0|             0|             0|
+---+---+---+--------+------+------+-------+-----------+--------------+--------------+



In [ ]:
# Combining all independent features
featuresassembler=VectorAssembler(inputCols=["age","bmi","children","sex_indexed","smoker_indexed","region_indexed"],outputCol="Independent Features")

In [ ]:
output=featuresassembler.transform(df1)
output.show()

+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+--------------------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_indexed|smoker_indexed|region_indexed|Independent Features|
+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+--------------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|        1.0|           1.0|           2.0|[19.0,27.9,0.0,1....|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|        0.0|           0.0|           0.0|[18.0,33.77,1.0,0...|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|        0.0|           0.0|           0.0|[28.0,33.0,3.0,0....|
| 33|  male|22.705|       0|    no|northwest|21984.47061|        0.0|           0.0|           1.0|[33.0,22.705,0.0,...|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|        0.0|           0.0|           1.0|[32.0,28.88,0.0,0...|
| 31|female| 25.74|       0|    

In [ ]:
# Independent Features
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[19.0,27.9,0.0,1....|
|[18.0,33.77,1.0,0...|
|[28.0,33.0,3.0,0....|
|[33.0,22.705,0.0,...|
|[32.0,28.88,0.0,0...|
|[31.0,25.74,0.0,1...|
|[46.0,33.44,1.0,1...|
|[37.0,27.74,3.0,1...|
|[37.0,29.83,2.0,0...|
|[60.0,25.84,0.0,1...|
|[25.0,26.22,0.0,0...|
|[62.0,26.29,0.0,1...|
|[23.0,34.4,0.0,0....|
|[56.0,39.82,0.0,1...|
|[27.0,42.13,0.0,0...|
|[19.0,24.6,1.0,0....|
|[52.0,30.78,1.0,1...|
|[23.0,23.845,0.0,...|
|[56.0,40.3,0.0,0....|
|[30.0,35.3,0.0,0....|
+--------------------+
only showing top 20 rows



In [ ]:
output.columns

['age',
 'sex',
 'bmi',
 'children',
 'smoker',
 'region',
 'charges',
 'sex_indexed',
 'smoker_indexed',
 'region_indexed',
 'Independent Features']

In [ ]:
final_data=output.select("Independent Features","Charges")

In [ ]:
final_data.show()

+--------------------+-----------+
|Independent Features|    Charges|
+--------------------+-----------+
|[19.0,27.9,0.0,1....|  16884.924|
|[18.0,33.77,1.0,0...|  1725.5523|
|[28.0,33.0,3.0,0....|   4449.462|
|[33.0,22.705,0.0,...|21984.47061|
|[32.0,28.88,0.0,0...|  3866.8552|
|[31.0,25.74,0.0,1...|  3756.6216|
|[46.0,33.44,1.0,1...|  8240.5896|
|[37.0,27.74,3.0,1...|  7281.5056|
|[37.0,29.83,2.0,0...|  6406.4107|
|[60.0,25.84,0.0,1...|28923.13692|
|[25.0,26.22,0.0,0...|  2721.3208|
|[62.0,26.29,0.0,1...| 27808.7251|
|[23.0,34.4,0.0,0....|   1826.843|
|[56.0,39.82,0.0,1...| 11090.7178|
|[27.0,42.13,0.0,0...| 39611.7577|
|[19.0,24.6,1.0,0....|   1837.237|
|[52.0,30.78,1.0,1...| 10797.3362|
|[23.0,23.845,0.0,...| 2395.17155|
|[56.0,40.3,0.0,0....|  10602.385|
|[30.0,35.3,0.0,0....|  36837.467|
+--------------------+-----------+
only showing top 20 rows



# Modelling

In [ ]:
# Splitting the data 
(train_df,test_df) =final_data.randomSplit([0.7,0.3])

In [ ]:
train_df.count()

911

In [ ]:
test_df.count()

427

## Linear Regression

In [ ]:
regressor = LinearRegression(featuresCol='Independent Features',labelCol='Charges')
regressor=regressor.fit(train_df)

In [ ]:
# Coefficients
regressor.coefficients

DenseVector([241.7805, 377.1683, 527.4263, 187.4761, 23647.491, 370.2141])

In [ ]:
# Intercept
regressor.intercept

-13651.92905147139

In [ ]:
# Prediction
pred = regressor.evaluate(test_df)

In [ ]:
# Results comparison
pred.predictions.show()

+--------------------+-----------+------------------+
|Independent Features|    Charges|        prediction|
+--------------------+-----------+------------------+
|(6,[0,1],[18.0,33...|  1136.3994|3395.6027921129826|
|(6,[0,1],[18.0,34...|   1137.011|  3561.55682219741|
|(6,[0,1],[18.0,41...|  1146.7966| 6216.821303548226|
|(6,[0,1],[18.0,53...|  1163.4627|10739.068623348834|
|(6,[0,1],[20.0,33...|  1391.5287| 3754.698218674752|
|(6,[0,1],[23.0,32...|  1824.2854| 4189.620089714641|
|(6,[0,1],[23.0,41...|  1837.2819| 7716.143229008689|
|(6,[0,1],[25.0,25...|  2137.6536|2100.8935725311258|
|(6,[0,1],[26.0,35...|  2322.6218|  5993.66270895104|
|(6,[0,1],[28.0,38...|  2689.4954| 7472.947838582684|
|(6,[0,1],[29.0,27...|  2867.1196| 3897.785621203435|
|(6,[0,1],[33.0,30...|  3704.3545| 5736.166177396844|
|(6,[0,1],[36.0,29...|   4399.731| 6254.065063478945|
|(6,[0,1],[40.0,25...|  5415.6612| 5478.669645842649|
|(6,[0,1],[40.0,41...|  5438.7491|11743.434281529724|
|(6,[0,1],[48.0,40...|  7804

In [ ]:
# Performance Metrics
pred.r2,pred.meanAbsoluteError,pred.meanSquaredError

(0.7521245440901827, 4001.4111035589503, 32885261.587168045)